# Custom User-based Model
The present notebooks aims at creating a UserBased class that inherits from the Algobase class (surprise package) and that can be customized with various similarity metrics, peer groups and score aggregation functions. 

In [1]:
# reloads modules automatically before entering the execution of code
#%load_ext autoreload
#%autoreload 2

# standard library imports
# -- add new imports here --

# third parties imports
import numpy as np 
import pandas as pd
from surprise import AlgoBase
# -- add new imports here --

# local imports
from constants import Constant as C
from loaders import load_ratings
# -- add new imports here --
from surprise import Dataset, KNNWithMeans
import matplotlib.pyplot as plt
import seaborn as sns

###
import heapq


# 1. Loading Data
Prepare a dataset in order to help implementing a user-based recommender system

In [4]:
# -- load data, build trainset and anti testset --
ratings = load_ratings(surprise_format=True)
trainset = ratings.build_full_trainset()
anti_testset = trainset.build_anti_testset()

# 2. Explore Surprise's user-based algorithm
Displays user-based predictions and similarity matrix on the test dataset using the KNNWithMeans class

In [ ]:
# -- using surprise's user-based algorithm, explore the impact of different parameters and displays predictions --
sim_options = {
    'name': 'msd',  # Mean Squared Difference
    'user_based': True,  # Modèle basé sur les utilisateurs
    'min_support': 3  # Minimum de 3 évaluations communes
}
# Créer une instance de KNNWithMeans
algo = KNNWithMeans(k=3, min_k=2, sim_options=sim_options)
# Entraîner le modèle
algo.fit(trainset)
# Faire une prédiction pour l'utilisateur 11 et l'élément 364
prediction = algo.predict(uid=11, iid=364)
print(f"Prédiction pour l'utilisateur 11 et l'élément 364 : {prediction.est}")


predictions = algo.test(anti_testset)
for pred in predictions[:30]:
    print(f"Utilisateur {pred.uid} a évalué l'élément {pred.iid} avec une note de {pred.est:.2f} (réel : {pred.r_ui}, actual_k = {pred.details.get('actual_k', 'N/A')})")
#1.La valeur de min_support est fixée à 3, ce qui signifie que pour qu'un voisin soit pris en compte dans le calcul de la prédiction il doit avoir au moins 3 évaluations communes avec l'utilisateur cible. Cela peut réduire le nombre de voisins valides et donc influencer la prédiction finale.
#2.Quand min_support est fixé à 3, la valeur de actual_k diminue pour certaines prédictions. Cela est dû au fait que actual_k représente le nombre de voisins qui ont réellement été utilisés pour calculer la prédiction et l'augmentation de min_support réduit le nombre de voisins valides

# Afficher une partie de la matrice de similarité (exemple pour les 10 premiers utilisateurs)
#print("\n--- Aperçu de la matrice de similarité utilisateur-utilisateur ---")
#sim_matrix = algo.sim  # Matrice numpy carrée (n_users x n_users)

# Afficher une partie de la matrice de similarité (exemple pour les 10 premiers utilisateurs)
#print("\n--- Aperçu de la matrice de similarité utilisateur-utilisateur ---")
#sim_matrix = algo.sim  # Matrice numpy carrée (n_users x n_users)
#n_max = min(10, sim_matrix.shape[0])  # On ne dépasse pas la taille réelle
#for i in range(n_max):
 #   print(f"Similarités de l'utilisateur interne {i} avec les autres : {sim_matrix[i, :n_max]}")

Computing the msd similarity matrix...
Done computing similarity matrix.
Prédiction pour l'utilisateur 11 et l'élément 364 : 2.49203431372549
Utilisateur 11 a évalué l'élément 1214 avec une note de 3.60 (réel : 3.1333333333333333, actual_k = 1)
Utilisateur 11 a évalué l'élément 364 avec une note de 2.49 (réel : 3.1333333333333333, actual_k = 2)
Utilisateur 11 a évalué l'élément 4308 avec une note de 1.60 (réel : 3.1333333333333333, actual_k = 1)
Utilisateur 11 a évalué l'élément 527 avec une note de 3.90 (réel : 3.1333333333333333, actual_k = 2)
Utilisateur 13 a évalué l'élément 1997 avec une note de 2.80 (réel : 3.1333333333333333, actual_k = 0)
Utilisateur 13 a évalué l'élément 4993 avec une note de 3.24 (réel : 3.1333333333333333, actual_k = 1)
Utilisateur 13 a évalué l'élément 2700 avec une note de 2.80 (réel : 3.1333333333333333, actual_k = 0)
Utilisateur 13 a évalué l'élément 1721 avec une note de 1.24 (réel : 3.1333333333333333, actual_k = 1)
Utilisateur 13 a évalué l'élément 52

observations minK:

- observations minK 2 par rapport à 1 sont similaire ou légerement plus elevé :Cela s'explique par le fait que l’algorithme impose une faible  contraintes sur le nombre minimum de voisins requis pour effectuer une prédiction. Avec des valeurs faibles de min_k, comme 1 ou 2, les prédictions peuvent être faites même avec très peu de voisins ce qui explique la faible variation entre les deux cas.
Cependant, pour certaines prédictions où le nombre de voisins disponibles est inférieur à 2, l’algorithme utilise une valeur par défaut comme la moyenne des évaluations de l’utilisateur ou la moyenne globale.

- observation mink= 3: on augmente le nombre minimum de voisins requis pour faire une prédiction ce qui rend l’algorithme plus strict.
Dans les 30 premières prédictions observées la majorité des valeurs restent similaires mais plusieurs prédictions augmentent, notamment celle de l'utilisateur 11 pour l’élément 364. Cette hausse est due au fait qu’il n’y avait pas assez de voisins (moins de 3) pour faire une prédiction personnalisée. L’algorithme a donc utilisé une valeur par défaut par exemple (moyenne des évaluations de l'utilisateur ou la moyenne globale) ce qui peut expliquer l’augmentation.

observations min support: 

- Plus la valeur de min_support est élevée, plus actual_k a tendance à diminuer.Cela s'explique par le fait qu’un min_support plus grand impose un critère plus strict : chaque voisin doit avoir au moins ce nombre d’évaluations communes avec l’utilisateur cible (par exemple, min_support = 3 signifie au moins 3 évaluations communes).Ainsi, moins de voisins respectent cette condition  ce qui réduit le nombre de voisins utilisables pour la prédiction et donc reduit le actual_k. 

- La variable actual_k représente le nombre de voisins qui ont réellement contribué à la prédiction.Plus actual_k est élevé, plus la prédiction est considérée comme fiable, car elle repose sur un plus grand nombre d’avis pertinents.

# 3. Implement and explore a customizable user-based algorithm
Create a self-made user-based algorithm allowing to customize the similarity metric, peer group calculation and aggregation function

changement de sort a heapq 

erreur dans le calcul du msd (trouver avec chat l'erreur de calcul)

remplacement par np.nan  dans estimate 



In [71]:
class UserBased(AlgoBase):
    def __init__(self, k=3, min_k=1, sim_options={}, **kwargs):
        AlgoBase.__init__(self, sim_options=sim_options, **kwargs)
        self.k = k
        self.min_k = min_k

        
    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        # Calcul de la matrice des ratings
        self.compute_rating_matrix()
        # Calcul de la matrice de similarité
        self.compute_similarity_matrix()
        # Calcul de la moyenne des notes par utilisateur
        self.mean_ratings = []
        for u in range(self.trainset.n_users):
            user_ratings = []
            for (_, rating) in self.trainset.ur[u]: #_ correspond à l'index de l'item
                user_ratings.append(rating)
            if user_ratings:
                mean_rating = np.mean(user_ratings)
            else:
                mean_rating = float('nan')  # ou 0.0 si tu préfères éviter les NaN
            self.mean_ratings.append(mean_rating)

    
    def estimate(self, u, i):
            if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
                raise np.nan 
            
            estimate = self.mean_ratings[u]

            ## Obtenir les voisins de l'utilisateur u qui ont noté l'item i
            neighbors = []
            for (v, rating) in self.trainset.ir[i]:  
                if v == u:
                    continue  # ne pas se comparer à soi-même

                sim_uv = self.sim[u, v]  # similarité entre u et v

                if sim_uv > 0 and not np.isnan(self.ratings_matrix[v, i]):  # si la similarité est positive et que v a noté l'item i
                    mean_v = self.mean_ratings[v]  # moyenne des notes de v
                    neighbors.append((sim_uv, rating - mean_v))

            # Trier les voisins par similarité décroissante
            top_k_neighbors = heapq.nlargest(self.k, neighbors, key=lambda x: x[0])

            # Calcul de la moyenne pondérée sur les k meilleurs voisins
            actual_k = 0
            weighted_sum = 0.0
            sum_sim = 0.0

            for sim, rating_diff in top_k_neighbors:
                if actual_k == self.k:
                    break
                weighted_sum += sim * rating_diff
                sum_sim += sim
                actual_k += 1

            # Vérifier si on a suffisamment de voisins
            if actual_k >= self.min_k and sum_sim > 0:
                estimate += weighted_sum / sum_sim

            return estimate


                            
    def compute_rating_matrix(self):
        # -- implement here the compute_rating_matrix function --
        self.ratings_matrix = np.empty((self.trainset.n_users, self.trainset.n_items))
        self.ratings_matrix[:] = np.nan
        for u in range(self.trainset.n_users): # or each user
            for i, rating in self.trainset.ur[u]: #for each item rated by the user
                self.ratings_matrix[u, i] = rating

    
    def compute_similarity_matrix(self):
        m = self.trainset.n_users
        ratings_matrix = self.ratings_matrix
        min_support = self.sim_options.get('min_support', 1)
        sim_name = self.sim_options.get("name", "euclidean")  # valeur par défaut

        # Initialiser la matrice de similarité
        self.sim = np.eye(m)

        for i in range(m):
            for j in range(i + 1, m):  # j > i pour éviter les doublons
                row_i = ratings_matrix[i]
                row_j = ratings_matrix[j]

                if sim_name == "jaccard":
                    sim = self.jaccard_similarity(row_i, row_j)
                    support = np.sum(~np.isnan(row_i) & ~np.isnan(row_j))

                else:
                    # Par défaut : similarité euclidienne normalisée
                    diff = row_i - row_j
                    support = np.sum(~np.isnan(diff))
                    if support >= min_support:
                        msd = np.nanmean((diff[~np.isnan(diff)]) ** 2)
                        sim = 1 / (1 + msd)
                    else:
                        sim = 0

                if support >= min_support:
                    self.sim[i, j] = sim
                    self.sim[j, i] = sim

    def jaccard_similarity(self, row_i, row_j):
        # Masques binaires : True là où il y a une note
        mask_i = ~np.isnan(row_i)
        mask_j = ~np.isnan(row_j)

        intersection = np.sum(mask_i & mask_j)
        union = np.sum(mask_i | mask_j)

        if union == 0:
            return 0.0
        return intersection / union


# 4. Compare KNNWithMeans with UserBased
Try to replicate KNNWithMeans with your self-made UserBased and check that outcomes are identical

In [72]:
# -- assert that predictions are the same with different sim_options --
#comparaison entre requltat Kmeans et resultqts user based 
import surprise
from surprise import accuracy

# Paramètres de similarité
sim_options = {
    'name': 'msd',
    'user_based': True,
    'min_support': 3
}
k = 3
min_k = 2

# Ton algorithme custom
userbased_algo = UserBased(k=k, min_k=min_k, sim_options=sim_options)
userbased_algo.fit(trainset)

# Algo officiel de surprise
knn_algo = KNNWithMeans(k=k, min_k=min_k, sim_options=sim_options)
knn_algo.fit(trainset)

# Anti-testset (les notes absentes dans le trainset)
anti_testset = trainset.build_anti_testset()

# Comparer les 30 premières prédictions
print(f"{'UID':>5} {'IID':>5} {'KNN est.':>10} {'UserBased est.':>15} {'Diff':>10}")
print("-" * 50)

for i, (uid, iid, _) in enumerate(anti_testset[:30]):
    pred_knn = knn_algo.predict(uid, iid)
    pred_userbased = userbased_algo.predict(uid, iid)

    diff = abs(pred_knn.est - pred_userbased.est)

    print(f"{uid:>5} {iid:>5} {pred_knn.est:10.4f} {pred_userbased.est:15.4f} {diff:10.4f}")



Computing the msd similarity matrix...
Done computing similarity matrix.
  UID   IID   KNN est.  UserBased est.       Diff
--------------------------------------------------
   11  1214     3.1667          3.1667     0.0000
   11   364     2.4920          2.4920     0.0000
   11  4308     3.1667          3.1667     0.0000
   11   527     3.8989          3.8989     0.0000
   13  1997     2.8000          2.8000     0.0000
   13  4993     2.8000          2.8000     0.0000
   13  2700     2.8000          2.8000     0.0000
   13  1721     2.8000          2.8000     0.0000
   13   527     2.8000          2.8000     0.0000
   17  2028     3.8125          3.8125     0.0000
   17  4993     4.1283          4.1283     0.0000
   17  1214     3.2500          3.2500     0.0000
   17  4308     3.2500          3.2500     0.0000
   19  1997     3.5000          3.5000     0.0000
   19  2028     3.5000          3.5000     0.0000
   19  4993     3.5000          3.5000     0.0000
   19  5952     3.5000    

# 5. Compare MSD and Jacard
Compare predictions made with MSD similarity and Jacard similarity


In [ ]:
# -- compare predictions made with MSD similarity and Jacard similarity --
# -- compare predictions made with MSD similarity and Jacard similarity --
sim_options = {"name": "jacard", "min_support": 1}
algo = UserBased(k=3, min_k=1, sim_options=sim_options)
algo.fit(trainset)
predictions_jacard = algo.test(anti_testset)
print(f"Prédictions avec la similarité Jacard : {predictions_jacard[:30]}")